In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('../datasrc/HIST_PAINEL_COVIDBR_20mai2020.xlsx')

In [3]:
df.head(5)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,0,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,0,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-01,10,210147125.0,2,0,NaN,NaN


In [4]:
df.columns

Index(['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude',
       'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019',
       'casosAcumulado', 'obitosAcumulado', 'Recuperadosnovos',
       'emAcompanhamentoNovos'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95916 entries, 0 to 95915
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   regiao                 95916 non-null  object 
 1   estado                 95831 non-null  object 
 2   municipio              93451 non-null  object 
 3   coduf                  95916 non-null  int64  
 4   codmun                 93989 non-null  float64
 5   codRegiaoSaude         93451 non-null  float64
 6   nomeRegiaoSaude        93451 non-null  object 
 7   data                   95916 non-null  object 
 8   semanaEpi              95916 non-null  int64  
 9   populacaoTCU2019       95378 non-null  float64
 10  casosAcumulado         95916 non-null  int64  
 11  obitosAcumulado        95916 non-null  int64  
 12  Recuperadosnovos       32 non-null     float64
 13  emAcompanhamentoNovos  32 non-null     float64
dtypes: float64(5), int64(4), object(5)
memory usage: 10.2+

In [6]:
df['estado'].unique()

array([nan, 'RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE',
       'RN', 'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR',
       'SC', 'RS', 'MS', 'MT', 'GO', 'DF'], dtype=object)

## Divide as base de dados (cidades e estados)

Primeiramente vamos extrair a base apenas de municípios.

In [7]:
df_city = df[df['municipio'].notnull()]

In [8]:
df_city = df_city.fillna(0)

In [9]:
df_city.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
1930,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-02,18,22945.0,1,0,0.0,0.0
1931,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-03,19,22945.0,1,0,0.0,0.0
1932,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-04,19,22945.0,1,0,0.0,0.0
1933,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-05,19,22945.0,1,0,0.0,0.0
1934,Norte,RO,Alta Floresta D'Oeste,11,110001.0,11005.0,Zona da Mata,2020-05-06,19,22945.0,1,0,0.0,0.0


Agora vamos extrair a base de estados apenas com as colunas de interesse.

In [10]:
df_state = df[(df['municipio'].isnull()) & (df['estado'].notnull())]

In [11]:
df_state = df_state.drop(['municipio', 
                          'codmun', 
                          'codRegiaoSaude', 
                          'nomeRegiaoSaude', 
                          'emAcompanhamentoNovos'], axis=1)

df_state = df_state.fillna(0)

In [12]:
df_state.head()

,regiao,estado,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos
85,Norte,RO,11,2020-03-20,12,1777225.0,1,0,0.0
86,Norte,RO,11,2020-03-21,12,1777225.0,1,0,0.0
87,Norte,RO,11,2020-03-22,13,1777225.0,3,0,0.0
88,Norte,RO,11,2020-03-23,13,1777225.0,3,0,0.0
89,Norte,RO,11,2020-03-24,13,1777225.0,3,0,0.0


In [13]:
df_country = df[(df['municipio'].isnull()) & (df['estado'].isnull())]

In [14]:
df_country = df_country.drop(['municipio', 
                              'codmun', 
                              'estado',
                              'codRegiaoSaude', 
                              'nomeRegiaoSaude', 
                              'emAcompanhamentoNovos'], axis=1)

df_country = df_country.fillna(0)
df_country['casosPorDia'] = df_country['casosAcumulado'].diff().fillna(0)
df_country['obitosPorDia'] = df_country['obitosAcumulado'].diff().fillna(0)

In [15]:
df_country.head()

,regiao,coduf,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,casosPorDia,obitosPorDia
0,Brasil,76,2020-02-26,9,210147125.0,1,0,0.0,0.0,0.0
1,Brasil,76,2020-02-27,9,210147125.0,1,0,0.0,0.0,0.0
2,Brasil,76,2020-02-28,9,210147125.0,1,0,0.0,0.0,0.0
3,Brasil,76,2020-02-29,9,210147125.0,2,0,0.0,1.0,0.0
4,Brasil,76,2020-03-01,10,210147125.0,2,0,0.0,0.0,0.0


Vamos salvar os dataframes em arquivo excel para leitura no PowerBI.

In [19]:
df_city.to_excel('../datasets/dataset_covid_city.xlsx')

In [20]:
df_state.to_excel('../datasets/dataset_covid_state.xlsx')

In [21]:
df_country.to_excel('../datasets/dataset_covid_country.xlsx')

## Sandbox

In [26]:
df_city[df_city['municipio']=='Castanhal']

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
6526,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-03-28,13,200793.0,1,0,0.0,0.0
6527,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-03-29,14,200793.0,1,0,0.0,0.0
6528,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-03-30,14,200793.0,1,0,0.0,0.0
6529,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-03-31,14,200793.0,1,0,0.0,0.0
6530,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-04-01,14,200793.0,1,0,0.0,0.0
6531,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-04-02,14,200793.0,1,0,0.0,0.0
6532,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-04-03,14,200793.0,1,0,0.0,0.0
6533,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-04-04,14,200793.0,2,0,0.0,0.0
6534,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-04-05,15,200793.0,2,0,0.0,0.0
6535,Norte,PA,Castanhal,15,150240.0,15008.0,Metropolitana III,2020-04-06,15,200793.0,2,0,0.0,0.0
